In [20]:
%matplotlib auto
import numpy as np
import cv2
from keras.preprocessing.image import ImageDataGenerator

from skimage import io
import os
from skimage.filters import threshold_yen
from skimage.exposure import rescale_intensity
from skimage.io import imread, imsave
from matplotlib import pyplot as plt 

from PIL import Image
import imutils

Using matplotlib backend: Qt5Agg


In [28]:
import requests
address = 'http://192.168.0.30:8080/shot.jpg'
colors = {}
ohms_list = []

color_bands = {"Black": 0, "Brown": 1, "Red": 2, "Orange": 3, "Yellow": 4, "Green": 5, "Blue": 6, "Violet": 7, "Silver": 8, "White": 9}
color_multipliers = {"Black": 1, "Brown": 10, "Red": 100, "Orange": 1000, "Yellow": 10000, "Green": 100000, "Blue":1000000 , "Violet": 10000000, "Silver": 100000000, "White": 100000000}

temp_start = (900, 545)
temp_end = (1000, 550)

#Find the area that is bigger than the area threshold and store the color and its x-coordinate
def find_color(contour, color):
    for c in contour:
        area = cv2.contourArea(c)
        if area > area_limit:
            print(color, area)
            M = cv2.moments(c)
            cx = int(M["m10"]/ M["m00"])
            cy = int(M["m01"]/ M["m00"])
            x,y,w,h = cv2.boundingRect(c)
            cv2.rectangle(frame,(x,y),(x+w,y+h),(255,255,0),1)
            print("x: ", x)
            if x not in colors:
                colors[x] = color

    
                
                
while True:
        
    img_resp = requests.get(address)
    img_arr = np.array(bytearray(img_resp.content), dtype=np.uint8)
    frame = cv2.imdecode(img_arr, -1)
    #frame = cv2.imread(input)
    height, width, channels = frame.shape
    #start = (int(width/2 - 50), int(height/2))
    #end = (int(width/2 + 50),int(height/2) + 30)
    show_frame = frame
    
    cv2.bilateralFilter(frame, 5, 800, 800)
    
    cv2.rectangle(frame, temp_start, temp_end, (255,255,255), 1)
    
    frame = frame[550:560, 900:1000]
    
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    

    mask_blue = cv2.inRange(hsv, lower_blue, upper_blue)
    mask_green = cv2.inRange(hsv, lower_green, upper_green)
    mask_red = cv2.inRange(hsv, lower_red, upper_red)
    mask_yellow = cv2.inRange(hsv, lower_yellow, upper_yellow)
    mask_orange = cv2.inRange(hsv, lower_orange, upper_orange)
    mask_violet = cv2.inRange(hsv, lower_violet, upper_violet)
    mask_black = cv2.inRange(hsv, lower_black, upper_black)
    mask_brown = cv2.inRange(hsv, lower_brown, upper_brown)
    mask_silver = cv2.inRange(hsv, lower_silver, upper_silver)
    
    
    contour1 = cv2.findContours(mask_blue, cv2.RETR_TREE, cv2.CHAIN_APPROX_TC89_L1)
    contour1 = imutils.grab_contours(contour1)
    
    contour2 = cv2.findContours(mask_green, cv2.RETR_TREE, cv2.CHAIN_APPROX_TC89_L1)
    contour2 = imutils.grab_contours(contour2)
    
    contour3 = cv2.findContours(mask_red, cv2.RETR_TREE, cv2.CHAIN_APPROX_TC89_L1)
    contour3 = imutils.grab_contours(contour3)
    
    contour4 = cv2.findContours(mask_yellow, cv2.RETR_TREE, cv2.CHAIN_APPROX_TC89_L1)
    contour4 = imutils.grab_contours(contour4)
    
    contour5 = cv2.findContours(mask_orange, cv2.RETR_TREE, cv2.CHAIN_APPROX_TC89_L1)
    contour5 = imutils.grab_contours(contour5)
    
    contour6 = cv2.findContours(mask_violet, cv2.RETR_TREE, cv2.CHAIN_APPROX_TC89_L1)
    contour6 = imutils.grab_contours(contour6)
    
    contour7 = cv2.findContours(mask_black, cv2.RETR_TREE, cv2.CHAIN_APPROX_TC89_L1)
    contour7 = imutils.grab_contours(contour7)
    
    contour8 = cv2.findContours(mask_brown, cv2.RETR_TREE, cv2.CHAIN_APPROX_TC89_L1)
    contour8 = imutils.grab_contours(contour8)
    
    contour9 = cv2.findContours(mask_silver, cv2.RETR_TREE, cv2.CHAIN_APPROX_TC89_L1)
    contour9 = imutils.grab_contours(contour9)
    
    
    area_limit = 20
                
    find_color(contour1, 'Blue')
    find_color(contour2, 'Green')    
    find_color(contour3, 'Red')
    find_color(contour4, 'Yellow')
    find_color(contour5, 'Orange')
    find_color(contour6, 'Violet')
    find_color(contour7, 'Black')
    find_color(contour8, 'Brown')
    find_color(contour9, 'Silver')
    
    
    #Calculate and display ohmic value
    colors_list = []
    for color in colors.keys():
             colors_list.append(color)

    colors_list = sorted(colors_list)
    print(colors_list, " sorted and before pop")
    
    
    #If two of the same colors are almost on the same x-position, chose only one.
    #Remove colors that are near the same X-coordinate (e.g. if there are two 
    #colors detected on the same stripe)
    
     
    for i in range(len(colors_list)):
        try:
            if colors_list[i] - (colors_list[i+1]) < 0:
                print("reached if pop")
                test = colors_list.pop(i+1)
                print("popped: ", test)

               
        except IndexError:
            print("index error")
            
    print(colors_list, " sorted and after pop")
    
    try:
        if len(colors_list) <3:
            print("reached if")
            pass

        else:
            color1 = colors[colors_list[0]]
            color2 = colors[colors_list[1]]
            color3 = colors[colors_list[2]]
            print("Colors: ", color1, color2, " * ", color3)
            colors = {}
            
            
        combined = int(str(color_bands[color1]) + str(color_bands[color2]))

        ohms = combined*color_multipliers[color3]

        if ohms < 10**3:
            ohms = (combined*color_multipliers[color3])
            disp = str(ohms)
        if ohms < 10**6 and ohms > 10**3:
            ohms = (combined*color_multipliers[color3])/(10**3)
            disp = str(ohms) + "k"
        if ohms >= 10**6:
            ohms = (combined*color_multipliers[color3])/(10**6)
            disp = str(ohms) + "M"
        
        
            #Filter out value noise, list -> most frequent element gets printed to reduce twitching    
        ohms_list.append(disp)

        if len(ohms_list) > 40:
            print("list reset")
            ohms_list = [ohms]

        if disp == '0':
            print("skip 0's")
            pass

        else:
            disp = str(max(ohms_list, key=ohms_list.count))
            print(ohms_list) 

        disp_colors = str("Colors: " + color1 + " " + color2 + " * " + color3)

        cv2.putText(show_frame,disp + ' ohm',(0,int(height*0.9)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        print("output: " + disp)
        print("_______________________")
        cv2.putText(show_frame,disp_colors,(int(width*0.75),int(height*0.9)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)


    except NameError:
        print("NameError")
                
    
    print(show_frame.shape)
    cv2.imshow("result",show_frame)
    
    #Close window
    key = cv2.waitKey(1) 
    if key == 27:
        break
    
        print(colors)
    
        

cv2.destroyAllWindows()
cv2.waitKey(1)

Brown 440.0
x:  44
[44]  sorted and before pop
index error
[44]  sorted and after pop
reached if
['10']
output: 10
_______________________
(1080, 1920, 3)
[44]  sorted and before pop
index error
[44]  sorted and after pop
reached if
['10', '10']
output: 10
_______________________
(1080, 1920, 3)
Orange 792.0
x:  0
[0, 44]  sorted and before pop
reached if pop
popped:  44
index error
[0]  sorted and after pop
reached if
['10', '10', '10']
output: 10
_______________________
(1080, 1920, 3)
Brown 792.0
x:  0
[0, 44]  sorted and before pop
reached if pop
popped:  44
index error
[0]  sorted and after pop
reached if
['10', '10', '10', '10']
output: 10
_______________________
(1080, 1920, 3)
Brown 405.5
x:  0
[0, 44]  sorted and before pop
reached if pop
popped:  44
index error
[0]  sorted and after pop
reached if
['10', '10', '10', '10', '10']
output: 10
_______________________
(1080, 1920, 3)
[0, 44]  sorted and before pop
reached if pop
popped:  44
index error
[0]  sorted and after pop
rea

[0, 44]  sorted and before pop
reached if pop
popped:  44
index error
[0]  sorted and after pop
reached if
[10, '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10']
output: 10
_______________________
(1080, 1920, 3)
[0, 44]  sorted and before pop
reached if pop
popped:  44
index error
[0]  sorted and after pop
reached if
[10, '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10']
output: 10
_______________________
(1080, 1920, 3)
[0, 44]  sorted and before pop
reached if pop
popped:  44
index error
[0]  sorted and after pop
reached if
list reset
[10]
output: 10
_______________________
(1080, 1920, 3)
Black 37.0
x:  82
Brown 79.0
x:  48
[0, 44, 48, 82]  sorted and before pop
reached if pop
popped:  44
reached if pop
popped:  82
index error
index error
[0, 48]  sorted and after pop
reached if
[10, '10']
output: 10
_______________________
(1080, 1920, 3)
[0, 44, 48, 82]  sorte

output: 1.1M
_______________________
(1080, 1920, 3)
Red 30.5
x:  28
Yellow 25.5
x:  12
Black 46.0
x:  58
Brown 39.0
x:  32
Brown 29.0
x:  0
[0, 12, 28, 32, 58]  sorted and before pop
reached if pop
popped:  12
reached if pop
popped:  32
index error
index error
index error
[0, 28, 58]  sorted and after pop
Colors:  Brown Red  *  Black
[1.2, '1.1M', '1.1M', '11', '11', '1.0M', '12']
output: 1.1M
_______________________
(1080, 1920, 3)
Black 46.0
x:  57
Brown 78.0
x:  26
[26, 57]  sorted and before pop
reached if pop
popped:  57
index error
[26]  sorted and after pop
reached if
[1.2, '1.1M', '1.1M', '11', '11', '1.0M', '12', '12']
output: 1.1M
_______________________
(1080, 1920, 3)
Green 29.5
x:  74
Brown 31.0
x:  23
[23, 26, 57, 74]  sorted and before pop
reached if pop
popped:  26
reached if pop
popped:  74
index error
index error
[23, 57]  sorted and after pop
reached if
[1.2, '1.1M', '1.1M', '11', '11', '1.0M', '12', '12', '12']
output: 12
_______________________
(1080, 1920, 3)
Bro

Brown 42.5
x:  46
[19, 39, 46]  sorted and before pop
reached if pop
popped:  39
index error
index error
[19, 46]  sorted and after pop
reached if
[11, '11', '1.1M', '1.1M', '1.1M', '1.1M', '1.0M', '1.0M', '1.0M', '1.0M', '1.0M', '1.0M', '1.0M', '1.0M']
output: 1.0M
_______________________
(1080, 1920, 3)
[19, 39, 46]  sorted and before pop
reached if pop
popped:  39
index error
index error
[19, 46]  sorted and after pop
reached if
[11, '11', '1.1M', '1.1M', '1.1M', '1.1M', '1.0M', '1.0M', '1.0M', '1.0M', '1.0M', '1.0M', '1.0M', '1.0M', '1.0M']
output: 1.0M
_______________________
(1080, 1920, 3)
Black 48.0
x:  86
Brown 28.0
x:  5
Brown 93.0
x:  52
[5, 19, 39, 46, 52, 86]  sorted and before pop
reached if pop
popped:  19
reached if pop
popped:  46
reached if pop
popped:  86
index error
index error
index error
[5, 39, 52]  sorted and after pop
Colors:  Brown Green  *  Brown
[11, '11', '1.1M', '1.1M', '1.1M', '1.1M', '1.0M', '1.0M', '1.0M', '1.0M', '1.0M', '1.0M', '1.0M', '1.0M', '1.0M',

Green 35.0
x:  54
Black 22.5
x:  32
Brown 85.5
x:  0
[0, 32, 54]  sorted and before pop
reached if pop
popped:  32
index error
index error
[0, 54]  sorted and after pop
reached if
[1.1, '1.1M', '1.1M', '11', '11', '11', '11', '1.1M', '1.1M', '1.1M', '1.1M', '1.1M', '1.0M', '1.0M', '1.0M', '1.0M', '1.0M', '1.0M', '1.0M', '1.0M']
output: 1.0M
_______________________
(1080, 1920, 3)
Green 57.0
x:  55
Black 61.0
x:  34
Brown 40.0
x:  7
[0, 7, 32, 34, 54, 55]  sorted and before pop
reached if pop
popped:  7
reached if pop
popped:  34
reached if pop
popped:  55
index error
index error
index error
[0, 32, 54]  sorted and after pop
Colors:  Brown Black  *  Green
list reset
[1.0]
output: 1.0
_______________________
(1080, 1920, 3)
Green 30.5
x:  58
Brown 44.0
x:  4
[4, 58]  sorted and before pop
reached if pop
popped:  58
index error
[4]  sorted and after pop
reached if
[1.0, '1.0M']
output: 1.0
_______________________
(1080, 1920, 3)
Green 38.0
x:  60
Brown 68.5
x:  6
[4, 6, 58, 60]  sorted an

...

Yellow 22.5
x:  89
Black 56.0
x:  52
Brown 84.5
x:  22
[21, 22, 52, 89]  sorted and before pop
reached if pop
popped:  22
reached if pop
popped:  89
index error
index error
[21, 52]  sorted and after pop
reached if
[1.1, '1.1M', '1.1M', '1.0M', '1.0M', '11', '11', '1.1M', '1.1M', '1.1M']
output: 1.1M
_______________________
(1080, 1920, 3)
Yellow 28.0
x:  40
Black 52.0
x:  51
Brown 90.0
x:  19
[19, 21, 22, 40, 51, 52, 89]  sorted and before pop
reached if pop
popped:  21
reached if pop
popped:  40
reached if pop
popped:  52
index error
index error
index error
index error
[19, 22, 51, 89]  sorted and after pop
Colors:  Brown Brown  *  Black
[1.1, '1.1M', '1.1M', '1.0M', '1.0M', '11', '11', '1.1M', '1.1M', '1.1M', '11']
output: 1.1M
_______________________
(1080, 1920, 3)
Red 27.0
x:  20
Yellow 24.0
x:  40
Black 59.0
x:  51
Brown 30.0
x:  27
[20, 27, 40, 51]  sorted and before pop
reached if pop
popped:  27
reached if pop
popped:  51
index error
index error
[20, 40]  sorted and after pop

Black 43.5
x:  56
Brown 52.0
x:  31
[31, 56]  sorted and before pop
reached if pop
popped:  56
index error
[31]  sorted and after pop
reached if
[11, '11', '10', '10', '10', '10', '10', '10', '10', '110', '110', '1.0M', '1.0M', '1.0M', '1.1M', '1.1M', '11', '11']
output: 10
_______________________
(1080, 1920, 3)
Green 33.0
x:  73
Black 62.0
x:  55
Brown 75.0
x:  26
[26, 31, 55, 56, 73]  sorted and before pop
reached if pop
popped:  31
reached if pop
popped:  56
index error
index error
index error
[26, 55, 73]  sorted and after pop
Colors:  Brown Black  *  Green
[11, '11', '10', '10', '10', '10', '10', '10', '10', '110', '110', '1.0M', '1.0M', '1.0M', '1.1M', '1.1M', '11', '11', '1.0M']
output: 10
_______________________
(1080, 1920, 3)
Green 35.5
x:  72
Black 54.0
x:  53
Brown 63.0
x:  24
[24, 53, 72]  sorted and before pop
reached if pop
popped:  53
index error
index error
[24, 72]  sorted and after pop
reached if
[11, '11', '10', '10', '10', '10', '10', '10', '10', '110', '110', '1.

Green 30.0
x:  55
Black 56.5
x:  35
Brown 86.5
x:  5
[5, 35, 55]  sorted and before pop
reached if pop
popped:  35
index error
index error
[5, 55]  sorted and after pop
reached if
[1.0, '1.0M', '11', '11']
output: 11
_______________________
(1080, 1920, 3)
[5, 35, 55]  sorted and before pop
reached if pop
popped:  35
index error
index error
[5, 55]  sorted and after pop
reached if
[1.0, '1.0M', '11', '11', '11']
output: 11
_______________________
(1080, 1920, 3)
[5, 35, 55]  sorted and before pop
reached if pop
popped:  35
index error
index error
[5, 55]  sorted and after pop
reached if
[1.0, '1.0M', '11', '11', '11', '11']
output: 11
_______________________
(1080, 1920, 3)
[5, 35, 55]  sorted and before pop
reached if pop
popped:  35
index error
index error
[5, 55]  sorted and after pop
reached if
[1.0, '1.0M', '11', '11', '11', '11', '11']
output: 11
_______________________
(1080, 1920, 3)
[5, 35, 55]  sorted and before pop
reached if pop
popped:  35
index error
index error
[5, 55]  

[8, 42, 61]  sorted and before pop
reached if pop
popped:  42
index error
index error
[8, 61]  sorted and after pop
reached if
[4.7, '4.7k', '47.0k', '47.0k', '4.7k', '4.7k', '4.7k', '47.0k', '47.0k', '4.7k', '4.7k', '4.7k']
output: 4.7k
_______________________
(1080, 1920, 3)
[8, 42, 61]  sorted and before pop
reached if pop
popped:  42
index error
index error
[8, 61]  sorted and after pop
reached if
[4.7, '4.7k', '47.0k', '47.0k', '4.7k', '4.7k', '4.7k', '47.0k', '47.0k', '4.7k', '4.7k', '4.7k', '4.7k']
output: 4.7k
_______________________
(1080, 1920, 3)
[8, 42, 61]  sorted and before pop
reached if pop
popped:  42
index error
index error
[8, 61]  sorted and after pop
reached if
[4.7, '4.7k', '47.0k', '47.0k', '4.7k', '4.7k', '4.7k', '47.0k', '47.0k', '4.7k', '4.7k', '4.7k', '4.7k', '4.7k']
output: 4.7k
_______________________
(1080, 1920, 3)
Silver 40.0
x:  4
[4, 8, 42, 61]  sorted and before pop
reached if pop
popped:  8
reached if pop
popped:  61
index error
index error
[4, 42]  

error: OpenCV(4.5.1) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-kh7iq4w7\opencv\modules\imgcodecs\src\loadsave.cpp:768: error: (-215:Assertion failed) !buf.empty() in function 'cv::imdecode_'


1.0M
